In [ ]:
#TRAFORMARE UN INDIRIZZO LEGGIBILE DALL'UOMO IN COORDINATE GEOGRAFICHE
import googlemaps
gmaps = googlemaps.Client(key='YOUR_API_KEY_HERE')
address = '1600 Amphitheatre Parkway, Mountain View, CA'
geocode_result = gmaps.geocode(address)
print(geocode_result[0]['geometry']['location'].values())

ValueError: Invalid API key provided.

In [46]:
#ANALIZZARE DATI SPAZIALI CON GEOPY E SHAPELY pag 27
import pandas as pd
from geopy.distance import distance
df = pd.read_csv('LOG.CSV', names = ['cab', 'lat', 'long', 'tm'])

#riordinimao le righe in base al'ultima posizione dei taxi, togiendo le righe più vecchie
latestrows = df.sort_values(['cab','tm'],ascending=False).drop_duplicates('cab')
print(latestrows)
latestrows = latestrows.values.tolist()

pick_up = 45.0862722712755, 7.66435708455579
#calcolare la distanza tra ogni taxi e il punto di raccolta
for i,row in enumerate(latestrows):
    dist = distance(pick_up, (row[1],row[2])).m
    print(row[0] + ':', round(dist))
    latestrows[i].append(round(dist))
#così determiniamo il taxi più vicino in base alla colonna 4(orario)
closest = min(latestrows, key=lambda x: x[4])
print('The closest cab is: ', closest[0], ' - the distance in meters: ', closest[4])

#questo è un calcolo approssimativo dato che calcoliamo la distanza in linea retta, ma non teniamo conto delle strade
#per calcolare in modo corretto la distanza tra due punti su una strada, possiamo usare la libreria SHAPELY
#che permette di calcolare la distanza tra un punto e una linea (una strada) o tra due punti su una strada usando poligoni e linee.
from shapely.geometry import Point, Polygon
coords = [(45.087431138440785, 7.651760766753362), (45.09623071049582, 7.657831250276273), (45.09514616189083, 7.671158204013811), (45.09056193206845, 7.669907888054073), (45.08251193328369, 7.666569904869294)]

poly = Polygon(coords)
taxi1 = Point(45.088178826955584, 7.657606107328147)
taxi2 = Point(45.0891585599824, 7.669495328334604)
pick_up = Point(45.09183288080605, 7.662181769671327)

print('taxi1 within the polygon:', taxi1.within(poly))
print('taxi2 within the polygon:', taxi2.within(poly))
print('pick_up within the polygon:', pick_up.within(poly))

#COMBINARE ENTRAMBI GLI APPROCCI
poly = Polygon(coords)
pick_up = Point(45.09183288080605, 7.662181769671327)
entry_point = Point(45.088178826955584, 7.657606107328147)

if taxi1.within(poly):
    dist1 = distance((pick_up.x, pick_up.y), (taxi1.x, taxi1.y)).m
else:
    dist1 = distance((taxi1.x, taxi1.y), (entry_point.x, entry_point.y)).m + distance((entry_point.x, entry_point.y), (pick_up.x, pick_up.y)).m

print('Distance from taxi1 to pick_up:', round(dist1))

           cab        lat      long        tm
6  CheFlanders  45.091371  7.661833  09:25:07
CheFlanders: 600
The closest cab is:  CheFlanders  - the distance in meters:  600
taxi1 within the polygon: True
taxi2 within the polygon: False
pick_up within the polygon: True
Distance from taxi1 to pick_up: 543


In [ ]:
#DERIVARE ATTRIBUTI NON SPAZIALI pag 35
import pandas as pd
orders = [
    ('order_039', 'open', 'cab_14'),
    ('order_034', 'open', 'cab_79'),
    ('order_032', 'open', 'cab_104'),
    ('order_026', 'closed', 'cab_79'),
    ('order_021', 'open', 'cab_45'),
    ('order_018', 'closed', 'cab_26'),
    ('order_008', 'closed', 'cab_112')
]

df_orders = pd.DataFrame(orders, columns=['order', 'status', 'cab'])
df_orders_open = df_orders[df_orders['status'] == 'open']
unvailable_list = df_orders_open['cab'].tolist()
print('Unavailable cabs:', unvailable_list)

#controllare quali di questi taxi disponibili sia più vicino al punto di raccolta
import geopy.distance as gd
pick_up = 46.083822, 38.967845 
cab_26 = 46.073852, 38.991890 
cab_112 = 46.078228, 39.003949 
cab_104 = 46.071226, 39.004947 
cab_14 = 46.004859, 38.095825 
cab_79 = 46.088621, 39.033929 
cab_45 = 46.141225, 39.124934 
cabs = {'cab_26': cab_26, 'cab_112': cab_112, 'cab_14': cab_14, 'cab_104': cab_104, 'cab_79': cab_79, 'cab_45': cab_45} 
dist_list = []

for cab_name, cab_loc in cabs.items():
    if cab_name not in unvailable_list:
        dist = distance(pick_up, cab_loc).m
        dist_list.append((cab_name, round(dist)))

print(dist_list)
print(min(dist_list, key=lambda x: x[1]))

Unavailable cabs: ['cab_14', 'cab_79', 'cab_104', 'cab_45']
[('cab_26', 2165), ('cab_112', 2861)]
('cab_26', 2165)


In [ ]:
#COMBINARE DATI SPAZIALI CON DATI NO SPAIALI
import pandas as pd
from geopy.distance import distance

cabs_list = [
('cab_14',1),
('cab_79',0),
('cab_104',0),
('cab_45',1),
('cab_26',0),
('cab_112',1)
]

df_cabs = pd.DataFrame(cabs_list, columns =['cab', 'seat'])
df_dist = pd.DataFrame(dist_list, columns =['cab', 'dist'])

df = pd.merge(df_cabs, df_dist, on='cab', how='inner')

result_list = list(df.itertuples(index=False,name=None))
result_list = [x for x in result_list if x[1] == 1]

print(min(result_list, key=lambda x: x[2]))

('cab_112', 1, 2861)
